## Carrega bibliotecas necessárias e a base de dados processada

In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv("../data/dataset_categorizado.csv")

## Qual a distribuição entre os sensíveis às camapanhas e não sensíveis?

In [15]:
subset = dataset.loc[dataset["Response"] == 1]
grupos = set(subset["grupo_cliente"].values)
tamanho = len(subset)

print("AVALIANDO A DISTRUIBUIÇÃO DE GRUPOS")
for grupo in grupos:
    ocorrencias = list(subset["grupo_cliente"].values).count(grupo)
    print(f"Grupo {grupo} {ocorrencias * 100 / tamanho} %")
print("")

campos = {
    #"MntWines": 0,
    "MntFruits": 0,
    #"MntMeatProducts": 0,
    "MntFishProducts": 0,
    "MntSweetProducts": 0,
    "MntGoldProds": 0
}

total_compras = 0
for campo in campos.keys():
    total = sum(subset[campo].values)
    campos[campo] = total
    total_compras = total_compras + total
    
print("AVALIANDO A DISTRUIBUIÇÃO DE PRODUTOS")
for campo in campos.keys():
    pp = campos[campo] * 100 / total_compras
    print(f"{campo}: {pp} %")

AVALIANDO A DISTRUIBUIÇÃO DE GRUPOS
Grupo 0 19.166666666666668 %
Grupo 1 21.666666666666668 %
Grupo 2 10.833333333333334 %
Grupo 3 22.5 %
Grupo 4 25.833333333333332 %

AVALIANDO A DISTRUIBUIÇÃO DE PRODUTOS
MntFruits: 16.99660942797769 %
MntFishProducts: 24.127748003937437 %
MntSweetProducts: 16.84348682051843 %
MntGoldProds: 42.03215574756644 %
